In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
def calc_price(phi1, phi2, cost, wtp):
    """compute each insurer's price (assumes interior solution)"""
    phi1, phi2 = phi1, phi2
    p1 = (2*phi1+phi2)/3. + cost
    p2 = (2*phi2+phi1)/3. + cost
    if phi1 <= 0 or phi2 <= 0:
        return 0,0
    
    return p1, p2
  
    
def calc_s(phi1, phi2, cost, wtp):
    """compute the proportion of people choosing each insurer
    assumes interior solution"""
    p1, p2 = calc_price(phi1, phi2, cost, wtp)
    s_hat = .5 + (p2 - p1)/(2.*cost)
    return s_hat


def calc_profits(phi1,phi2,cost, wtp, mc):
    phi1, phi2 = phi1, phi2
    s= calc_s(phi1, phi2, cost, wtp)
    p1,p2 = calc_price(phi1, phi2, cost, wtp)
    profits1, profits2 = s*(p1-phi1), (1-s)*(p2-phi2)
    mc1,mc2 = mc
    hosp_profit = s*(phi1-mc1) +  (1-s)*(phi2-mc2)
    
    if phi1 <= 0 or phi2 <= 0:
        return 0,0,0
    
    return hosp_profit, profits1, profits2

    
def outside_simlt(phi1, phi2, cost, wtp,  mc, active=False):
    """outside option in simult case
    assuming listprice high enough"""
    mc1,mc2 = mc
    
    if active:
        return (wtp-cost)/2 - mc2 #outside cost is other insurer?

    
    s_hat = calc_s(phi1, phi2, cost, wtp) # s_hat with no recapture
    p1,p2 = calc_price(phi1, phi2, cost, wtp)
    s_hat  = np.clip((cost + p2 - wtp)/cost,0,1) #s_hat with recapture
    
    
    return (1-s_hat)*(phi2)


calc_profits(5,0 ,5, 26, [0,0])

In [ ]:
#arbitrary outside option...
def nash_in_nash(phi1, phi2, cost, wtp, mc, beta=.5,outside=None):
    if outside == None:
        outside = outside_simlt(phi1, phi2,cost, wtp , mc, active=True)

    hosp_profit, profits1, profits2 = calc_profits(phi1, phi2,  cost,  wtp, mc)
    obj = -1*(np.log(max(hosp_profit-outside,1e-4))*(1-beta) 
              + np.log(profits1)*beta)
    return obj


def bargain_helper_seq(phi1, cost, wtp, mc,beta=.5):
    
    """ solve firm 1s optimization holding theta 2 fixed """
    outside = outside_simlt(phi1, None, cost, wtp, mc, active=True)
    result = minimize( nash_in_nash, 15., args=(phi1, cost, wtp, mc[::-1], beta, outside),
                      method='Nelder-Mead', options={'disp': False} )
    return result.x


def seq_obj(phi1, cost, wtp, mc, betas=np.array([.5,.5])):
    """ theta1 is a guess for the first stage """
    phi2 = bargain_helper_seq(phi1, cost, wtp, mc,betas[1])
    return nash_in_nash( phi1, phi2, cost, wtp, mc, betas[0], phi2 )


COST = 5
WTP = 25
MC = np.array([0,0])


plt.plot([i for i in np.linspace(COST+3,WTP +10,num=20)],  
         [seq_obj(i, 13.5 , WTP,MC) for i in np.linspace(COST+3,WTP +10,num=20)])

In [ ]:
#lower willingness to pay
COST = 5
WTP  = 25

print("v","phi 1" , "phi 2", "hosp profit", "profit 1", "profit 2")
for i in np.linspace(0,5,6):
    mc_i = np.array([0, 0])
    phi1,phi2 = seq_bargain(20,COST,WTP+i,mc_i)
    hosp_profit, profits1, profits2= calc_profits(phi1[0],phi2[0],COST, WTP+i, mc_i )
    p1,p2=calc_price(phi1, phi2, COST, WTP+i)
    s1 =calc_s(phi1, phi2, COST, WTP+i)
    print(i+WTP, round(phi1[0],3), round(phi2[0],3), round(hosp_profit,3), round(profits1,3), round(profits2,3),
         round(p1[0],3), round(p2[0],3),round(s1[0],3), round(1-s1[0],3))

In [ ]:
#generate liner solution

#just make sure it works for stuff i know is right

def compute_solution(active,nobs=20):
    #mcs = np.random.uniform(low=0.0, high=2.0, size=(2,nobs))
    phis = []
    vs = np.random.uniform(low=25,high=25.5,size=(nobs))
    lambdas = np.random.uniform(low=5,high=6,size=(nobs))
    x = np.concatenate(([vs],[lambdas])) #,mcs
    for i in range(nobs):
        phi = simult_bargain(11.25,11.25,lambdas[i],vs[i],np.array([0,0]),active=active) 
        phis.append(np.array(phi).flatten())
    phis = np.array(phis)
    xx = x.dot(x.transpose())
    xy1 = x.dot(phis[:,0])
    xy2 = x.dot(phis[:,1])
    return np.linalg.inv(xx).dot(xy1),np.linalg.inv(xx).dot(xy2)
    
print(compute_solution(active=True,nobs=10))
print(compute_solution(active=False,nobs=10))